In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np
from path import Path as path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.decomposition import PCA
import time
import numpy as np

In [103]:
class SmartRandomForest(RandomForestClassifier):
    
    def fit(self, X, Y, tgt = 0.8):
        components = round(np.array(X).shape[1]/2)
        pca = PCA(components)
        pca.fit(X)
        total = 0
        for i,e in enumerate(pca.explained_variance_ratio_):
            total += e
            if total >= tgt:
                n = i
                break
        self.pca = PCA(n)
        X2 = self.pca.fit_transform(X)
        return super().fit(X2, Y)
    
    def predict(self, X):
        X2 = self.pca.transform(X)
        return super().predict(X2)

In [4]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

d:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
imagenet = models.squeezenet1_1(pretrained=True)

d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [6]:
def PIL_to_raw(img_pil):
    img_tensor = preprocess(img_pil)
    img_tensor.unsqueeze_(0)
    img_variable = Variable(img_tensor)
    fc_out = imagenet(img_variable)
    return fc_out.data.numpy()

def pred_vector(img_pil):
    raw = PIL_to_raw(img_pil)
    preds = raw[0]
    result = np.array([np.exp(p) for p in preds])
    return result / sum(result)

In [7]:
mydir = "../../data/dogscats"

In [8]:
cat = 0
dog = 1

In [9]:
X_train, y_train, X_test, y_test = [],[],[],[]

In [10]:
for f in path(mydir + "/train/cats").files():
    X_train.append(pred_vector(Image.open(f)))
    y_train.append(cat)

In [11]:
for f in path(mydir + "/train/dogs").files():
    X_train.append(pred_vector(Image.open(f)))
    y_train.append(dog)

In [12]:
for f in path(mydir + "/valid/cats").files():
    X_test.append(pred_vector(Image.open(f)))
    y_test.append(cat)

In [13]:
for f in path(mydir + "/valid/dogs").files():
    X_test.append(pred_vector(Image.open(f)))
    y_test.append(dog)

In [14]:
X_train = np.array(X_train)

In [15]:
X_train.shape

(23000, 1000)

In [16]:
y_train = np.array(y_train)

In [17]:
y_train.shape

(23000,)

In [18]:
X_test = np.array(X_test)

In [19]:
y_test = np.array(y_test)

In [63]:
rf = RandomForestClassifier(n_estimators=50)

In [64]:
t0 = time.time()
rf.fit(X_train,y_train)
t1 = time.time()

In [65]:
y_pred = rf.predict(X_test)

In [66]:
f1_score(y_test, y_pred)

0.9606458123107972

In [67]:
accuracy_score(y_test, y_pred)

0.961

In [68]:
t1 - t0

30.893946170806885

In [104]:
sf = SmartRandomForest(n_estimators=100)

In [105]:
t2 = time.time()
sf.fit(X_train,y_train)
t3 = time.time()

In [106]:
t3 - t2


18.467889308929443

In [107]:
y_pred = sf.predict(X_test)

In [108]:
f1_score(y_test, y_pred)

0.944

In [109]:
accuracy_score(y_test, y_pred)

0.944